In [ ]:
import regex as re
from tqdm import tqdm

In [ ]:
data = """
initial state: #..#.#..##......###...###

...## => #
..#.. => #
.#... => #
.#.#. => #
.#.## => #
.##.. => #
.#### => #
#.#.# => #
#.### => #
##.#. => #
##.## => #
###.. => #
###.# => #
####. => #
""".strip().splitlines()


In [ ]:
# Make sure you create a file with the name below and save the real problem output there. 
# If you want to run just the sample data, skip this block
with open("./12-input.txt", "r") as FILE:
    data = FILE.read().strip().splitlines()

In [ ]:
pattern_initial = re.compile("initial state: ([\#\.]+)")
pattern_rule = re.compile("([\#\.]{5}) => ([\#\.])")

initial_state = pattern_initial.match(data[0])[1]
rules = {}
for rule_input in data[2:]:
    match = pattern_rule.match(rule_input)
    rules[match[1]] = match[2]

# We always want a string with zero being the centre pot,
# i.e. if the string is 20 long, then we want to add 19 empty
# pots before so that pot zero is always in the middle
initial_state = "." * (len(initial_state)-1) + initial_state

display(initial_state)
display(rules)

In [ ]:
def apply_rules(state):
    """
        Applies the rules to the given state
    """
    # If there is a pot in any of the four 'end' pots, then we need to extend 
    # the state to ensure we have a full generation
    if "#" in (state[0:2] + state[-3:-1]):
        state =  ".." + state + ".."
    
    # To properly consider the 'end' pots, we also have to add empty pots
    # but these are for evaluation only
    full_state = ".." + state + ".."
    result = []
    for pot in range(0, len(state)):
        # Assume empty pots either end
        segment = full_state[pot:pot+5]
        result.append(rules.get(segment,"."))
        
    return "".join(result)
        
apply_rules(initial_state)

In [ ]:
state=initial_state
print(state)
for gen in range(0,20):
    state = apply_rules(state)
    print(state)

In [ ]:
# We now score up the state
def score_plants(state):
    plant_sum = 0 
    for ix,pot in enumerate(state):
        pot_number = ix - int((len(state)-1)/2)
        if pot == "#":
            plant_sum += pot_number
    return plant_sum

score_plants(state)  

# Part 2

This is another optimisation problem. Running this for 50 billion generations would take... a while. 

Let's assume that since the question is framed like this, there will be a shortcut. Most obvious one, if you've ever programmed [game of life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) is the emergence of [stable patterns](https://en.wikipedia.org/wiki/Still_life_(cellular_automaton)).

Let's see if we simply see if any emerging repetitions appear.

In [ ]:
state=initial_state

# Let's check the first 10000 states to see if we have any
# patterns emerging
previous_states = set()
for gen in tqdm(range(0,100000)):
    state = apply_rules(state)
    # Remove empty plants from the ends
    plant_containing_state = state.strip(".")
    if plant_containing_state in previous_states:
        print("LOOP DETECTED IN GENERATION", gen)
        break
    else:
        previous_states.add(plant_containing_state)

In [ ]:
# So it looks like we have similar states, 
# are they in subsequent generations?

state=initial_state
last_state = ""
for gen in tqdm(range(0,100000)):
    state = apply_rules(state)
    # Remove empty plants from the ends
    plant_containing_state = state.strip(".")
    if plant_containing_state == last_state:
        print("LOOP DETECTED IN GENERATION", gen)
        break
    else:
        last_state = plant_containing_state


Perfect, in both the sample data, and my data, there are stable patterns emerging. Hopefully in yours as well.

Let's assume the patterns are 'drifting', so we just need to understand the drift. 



In [ ]:
state=initial_state
last_state = None
last_full_state = None
first_repeat_state = None
first_repeat_state_gen = None
for gen in tqdm(range(0,100000)):
    state = apply_rules(state)
    # Remove empty plants from the ends
    plant_containing_state = state.strip(".")
    if plant_containing_state == last_state:
        first_repeat_state = state
        first_repeat_state_gen = gen
        print("LOOP DETECTED IN GENERATION", first_repeat_state_gen)
        break
    else:
        last_state = plant_containing_state
        last_full_state = state

display(last_full_state)
display(first_repeat_state)

# Calculate the positions of the first plant containing pots for each state
state1_pos = last_full_state.index("#") - int((len(last_full_state)-1)/2)
state2_pos = first_repeat_state.index("#") - int((len(first_repeat_state)-1)/2)
drift = state2_pos - state1_pos

print("The drift of the plant containing pattern is", drift)




In [ ]:
# We now have what we need to figure out where the plants are going to be after fifty billion (50000000000) generations
plant_pattern = first_repeat_state.strip(".")
plant_pattern_start = state2_pos + (50000000000*drift) - first_repeat_state_gen

# Slightly different than before - this 
def score_plants_with_start_position(state, start_position):
    plant_sum = 0 
    for ix,pot in enumerate(state):
        pot_number = start_position + ix
        if pot == "#":
            plant_sum += pot_number
    return plant_sum

# To test we score up the full 'first repeat state' and compare it with the 'forwarded score' of just the plant containing part
# They should match
print("Score for first repeat state",score_plants(first_repeat_state))
print("Score for repeat pattern at repeat generation", score_plants_with_start_position(plant_pattern, state2_pos))

# The previous generation should have been
print("Score for original repeat state",score_plants(last_full_state))
print("Score for repeat pattern at previous generation", score_plants_with_start_position(plant_pattern, state2_pos-drift))



In [ ]:
# Let's test to make sure it works by playing forward 2500 generations
test_generation = 2500
state=initial_state
for gen in range(0,test_generation):
    state = apply_rules(state)
score_plants(state)

In [ ]:
# That should match our 'fast forwarded' pattern score
target_generation = test_generation - first_repeat_state_gen
target_position = state2_pos - 1 + target_generation * drift
score_plants_with_start_position(plant_pattern, target_position)

In [ ]:
# We can now calculate score at fifty billion generations
target_generation = 50000000000 - first_repeat_state_gen
target_position = state2_pos - 1 + target_generation * drift

score_plants_with_start_position(plant_pattern, target_position)